## This notebook shows the canonical mean to select some spots with a simple filter
* It is possible to interact with external libraries, have a look on the `init_diagram` notebook.
* To instanciate a `Diagram` allows you to apply efficiently filters based on image processing.
* They are several means to instanciate a `diagram` some of them are betters than overs.

In [1]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram

In [2]:
"""Define some utils"""

def select_items(criteria: torch.Tensor, threshold: float, nb_min: int) -> torch.Tensor:
    """Select a least `nb_min` items and also `criteria` >= `threshold`."""
    if len(criteria) <= nb_min:
        return torch.arange(len(criteria))
    indexs = torch.argsort(confidence, descending=True)  # the indexs from the best to the worse
    threshold = min(threshold, indexs[nb_min-1].item())
    return criteria >= threshold

## Set and take control of the global configuration parameters
* The cache is automaticaly managed, but you can control it a bit.

In [3]:
# manage context

from laueimproc.opti.manager import DiagramManager

manager = DiagramManager()
manager.verbose = True  # to display the operations in background
manager.ram_limit = 0.8  # trigger cache clear after 80% of ram full, default based on swapiness
manager.update()  # dereference unreachable diagrams

update tracked diagrams, 0 are freed, 0 remain


## Initialise the diagrams
* The different ways of initializing a diagram are described in detail in the `init_diagram` notebook.

In [4]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
# all_files = list(pathlib.Path("/data/visitor/a322855/bm32/20240221/RAW_DATA/Almardi/Almardi_map2DGemardi3_GOOD_0004/scan0002").iterdir())
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()
    # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

### Basic filtering

In [5]:
"""Sorted by intensity"""

for diagram in tqdm.tqdm(diagrams):
    intensities = diagram.compute_pxl_intensities()
    sorted_indexs = torch.argsort(intensities, descending=True)
    diagram.filter_spots(sorted_indexs, msg="sorted by intensities", inplace=True)
    # print(diagram)
    # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

100%|████████████████████████████████████████████████████| 100/100 [00:04<00:00, 24.35it/s]


### Fit spots roi with gaussians

In [7]:
for diagram in tqdm.tqdm(diagrams):
# for diagram in diagrams:
    mean, cov, infodict = diagram.fit_gaussian(photon_density=10900.0, tol=True, parallel=False)
    confidence = 3.0 * infodict["tol"]  # 99% confidence interval on pixel position
    selection = select_items(-confidence, -0.5, 10)  # 10 best and position +- 0.5pxl
    diagram.filter_spots(selection, msg="remove spots with uncertain position", inplace=True)
    if 0:
        print(diagram)
        # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

 12%|██████▏                                             | 12/100 [00:00<00:00, 107.81it/s]

torch.Size([5, 120, 2]) torch.Size([5, 12, 10])
torch.Size([15, 1410, 2]) torch.Size([15, 47, 30])
torch.Size([25, 2015, 2]) torch.Size([25, 65, 31])
torch.Size([22, 1450, 2]) torch.Size([22, 58, 25])
torch.Size([26, 1862, 2]) torch.Size([26, 49, 38])
torch.Size([16, 1927, 2]) torch.Size([16, 47, 41])
torch.Size([127, 1470, 2]) torch.Size([127, 49, 30])
torch.Size([177, 3393, 2]) torch.Size([177, 87, 39])
torch.Size([134, 2072, 2]) torch.Size([134, 56, 37])
torch.Size([99, 3225, 2]) torch.Size([99, 75, 43])
torch.Size([112, 1856, 2]) torch.Size([112, 64, 29])
torch.Size([189, 1840, 2]) torch.Size([189, 46, 40])
torch.Size([186, 2183, 2]) torch.Size([186, 59, 37])
torch.Size([157, 2035, 2]) torch.Size([157, 55, 37])
torch.Size([113, 897, 2]) torch.Size([113, 39, 23])
torch.Size([79, 1944, 2]) torch.Size([79, 72, 27])
torch.Size([110, 2054, 2]) torch.Size([110, 79, 26])
torch.Size([108, 1794, 2]) torch.Size([108, 39, 46])
torch.Size([228, 484, 2]) torch.Size([228, 22, 22])
torch.Size([12

 35%|██████████████████▌                                  | 35/100 [00:00<00:00, 89.93it/s]

torch.Size([139, 442, 2]) torch.Size([139, 26, 17])
torch.Size([880, 522, 2]) torch.Size([880, 58, 9])
torch.Size([327, 714, 2]) torch.Size([327, 34, 21])
torch.Size([1062, 336, 2]) torch.Size([1062, 24, 14])
torch.Size([375, 646, 2]) torch.Size([375, 34, 19])
torch.Size([414, 924, 2]) torch.Size([414, 44, 21])
torch.Size([461, 666, 2]) torch.Size([461, 37, 18])
torch.Size([483, 840, 2]) torch.Size([483, 40, 21])
torch.Size([437, 703, 2]) torch.Size([437, 37, 19])
torch.Size([337, 532, 2]) torch.Size([337, 28, 19])
torch.Size([1622, 375, 2]) torch.Size([1622, 25, 15])
torch.Size([400, 900, 2]) torch.Size([400, 36, 25])
torch.Size([455, 493, 2]) torch.Size([455, 29, 17])
torch.Size([1400, 286, 2]) torch.Size([1400, 22, 13])
torch.Size([662, 72, 2]) torch.Size([662, 9, 8])
torch.Size([0, 6020, 2]) torch.Size([0, 70, 86])


 37%|███████████████████▌                                 | 37/100 [00:00<00:00, 87.68it/s]


RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous